In [1]:
# Library header
from PIL import Image
from matplotlib import pyplot as plt
from pathlib import Path
from tensorflow.keras import Model, Input
from tqdm import tqdm
import shutil #library for moving files
import tensorflow as tf
import numpy as np
import cv2

# own libraries
import split_dataset as spData
import handseg_two_classes as hand
import data_gen as dg
import util as ut

#globals
input_width = 640
input_height = 480
handseg_path = '../../../handseg-150k'

0.2.9


In [2]:
# preprocessing
def preprocess(img):
    return (img / img.max()) * 255

# create the data generators with data_loader
# ... for training
train_gen = dg.image_segmentation_generator(
        images_path=handseg_path+"/images/",
        segs_path=handseg_path+"/masks/", 
        batch_size=32,
        n_classes=2,
        input_height=480,
        input_width=640,
        output_height=480,
        output_width=640,
        do_augment=False,
        preprocessing=preprocess,
        read_image_type=1)
# read_image_type -> 0 = grayscale; -> 1 = rgb

# ... for validation
val_gen = dg.image_segmentation_generator(
        images_path=handseg_path+"/val_images/",
        segs_path=handseg_path+"/val_masks/",
        batch_size=32,
        n_classes=2,
        input_height=480,
        input_width=640,
        output_height=480,
        output_width=640,
        preprocessing=preprocess,
        read_image_type=1)

# ... for testing
test_gen = dg.image_segmentation_generator(
        images_path=handseg_path+"/test_images/",
        segs_path=handseg_path+"/test_masks/",
        batch_size=2,
        n_classes=2,
        input_height=480,
        input_width=640,
        output_height=480,
        output_width=640,
        preprocessing=preprocess,
        read_image_type=1)

In [ ]:
# load the weights from previous training
handseg.load_weights("./handseg_model_twoClasses_4.h5")

# training
history = handseg.fit(x=train_gen,
            epochs=10,
            batch_size=32,
            steps_per_epoch=512,
            validation_data=val_gen,
            validation_batch_size=16,
            validation_steps=256)

# load/save the already trained weigths
handseg.save_weights("./handseg_model_twoClasses_updated.h5")

In [ ]:
# evaluation
# returns a list of tupels containing the values for iou and accuracy
# since the metrics are updated in each step, the las tupel value containst the
# final result
# load the weights from previous training
handseg.load_weights("../../../results/weights/twoClasses/handseg_model_twoClasses_4.h5")

def eval_handseg():
  # path values for test data
  paths_test_images = glob.glob(handseg_path+"/test_images/*.png")
  paths_test_mask = glob.glob(handseg_path+"/test_masks/*.png")

  # create the respective metrics
  metric_iou = tf.keras.metrics.MeanIoU(num_classes=2)
  metric_acc = tf.keras.metrics.Accuracy()

  # initialize list
  result = []

  for i in tqdm(range(len(paths_test_images))):
    # load img
    in_image = cv2.imread(paths_test_images[i])
    in_image = np.reshape(in_image, (1,480, 640, 3))
    # apply preprocessing
    in_image = ut.preprocess(in_image)
    # predict
    out_im = handseg.predict(in_image)
    # interpret prediction
    out_im = ut.interpretPrediction(out_im[0])
    # load mask image
    gt = cv2.imread(paths_test_mask[i])
    
    # apply preprocessing to mask images (3 -> 2 classes)
    gt = tf.where(gt == 2, 1, gt)
    
    # calculate IoU and accuracy
    metric_iou.update_state(gt[:,:,0], out_im)
    metric_acc.update_state(gt[:,:,0], out_im)
    # append tupel to list
    result.append((metric_iou.result().numpy(), metric_acc.result().numpy()))

    # reset the metric states
    metric_iou.reset_states()
    metric_acc.reset_states()

  return result

res = eval_handseg()
n = len(res)
sum_iou = 0
sum_acc = 0
for values in res:
    sum_iou += values[0]
    sum_acc += values[1]
mean_iou = sum_iou/n
mean_acc = sum_acc/n
print("Mean IoU: ", mean_iou)
print("Mean Acc: ", mean_acc)

In [ ]:
# these are helper functions for evaluation
# this function will create a segmentation and create the bounding box
def predictBox(img, id):
    #reshape input image
    input_image = cv2.resize(img, (640, 480))
    input_image = np.reshape(input_image, (1,480, 640, 3))
    # apply preprocessing
    input_image = preprocess2(input_image)
    # predict hand segmentation
    output_image = handseg.predict(input_image)
    # interpret prediction
    output_image = ut.interpretPrediction(output_image[0])
    # draw the bounding box
    box = drawBoxes_eval(output_image, img, id)
    return box

# get the bounding box values and save the image inside
def drawBoxes_eval(mask, img, id):
    # Iterate all colors in mask
    for color in np.unique(mask):

        # Color 0 is assumed to be background or artifacts
        if color == 0:
          continue

        # Determine bounding rectangle w.r.t. all pixels of the mask with
        # the current color
        x, y, w, h = cv2.boundingRect(np.uint8(mask == color))
        # Draw bounding rectangle to color image
        out = cv2.rectangle(img.copy(), (x, y), (x+w, y+h), (0, int(color), 255), 2)

        # Show image with bounding box
        #plt.imshow(out); plt.title('img_' + str(color)); plt.show()
        cv2.imwrite("../../../BoundingBox_eval/results/"+id+".png", out)
    return (x,y,w,h)

In [ ]:
# create eval for bounding box samples
# load weights
handseg.load_weights("../../../results/weights/twoClasses/handseg_model_twoClasses_4.h5")

# create eval directory
os.mkdir("../../../BoundingBox_eval")
# ... and respective subdirectories
os.mkdir("../../../BoundingBox_eval/test_seq_1")
os.mkdir("../../../BoundingBox_eval/test_seq_2")
os.mkdir("../../../BoundingBox_eval/results")

# initialize paths
sample_path_1 = "../../../ICVL/Depth/test_seq_1"
sample_path_2 = "../../../ICVL/Depth/test_seq_2"
eval_path_1 = "../../../BoundingBox_eval/test_seq_1"
eval_path_2 = "../../../BoundingBox_eval/test_seq_2"

#load all file names into a list
sample_paths_list_1 = glob.glob(sample_path_1+"/*.png")
sample_paths_list_2 = glob.glob(sample_path_2+"/*.png")

# shuffle paths and take first 200 samples
random.shuffle(sample_paths_list_1)
random.shuffle(sample_paths_list_2)

# move samples to the eval path
for i in range(100): 
    shutil.copy(sample_paths_list_1[i], eval_path_1)
    shutil.copy(sample_paths_list_2[i], eval_path_2)


# initialize list for evaluation samples
eval_paths_list_1 = glob.glob(eval_path_1+"/*.png")
eval_paths_list_2 = glob.glob(eval_path_2+"/*.png")

# collect all samples in eval_path result
for i in tqdm(range(100)):
    image_name_1 = eval_paths_list_1[i]
    img_1 = cv2.imread(image_name_1)
    image_name_2 = eval_paths_list_2[i]
    img_2 = cv2.imread(image_name_2)
    # predict bounding box via hand segmentation
    predictBox(img_1, str(i)+"_1")
    predictBox(img_2, str(i)+"_2")